### cohere modeli hfden upload etmeyi deniyoruz

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from transformers import AutoTokenizer, AutoModel
import os
from dotenv import load_dotenv

print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))
hf_token = os.getenv("HF_API_KEY")  

tokenizer = AutoTokenizer.from_pretrained(
    "Cohere/Cohere-embed-multilingual-v3.0", 
    token=hf_token
)
model = AutoModel.from_pretrained(
    "Cohere/Cohere-embed-multilingual-v3.0",
    token=hf_token,
    torch_dtype="auto"
)
# Örnek metinler
texts = ["Bu bir test cümlesidir.", "Mahkeme kararını verdi."]

# Tokenize et
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Modeli kullanarak embedding al
with torch.no_grad():
    outputs = model(**inputs)
    # Burada last_hidden_state'den mean pooling yapıyoruz
    embeddings = outputs.last_hidden_state.mean(dim=1)

print("Embedding boyutu:", embeddings.shape)
print("İlk embedding örneği:", embeddings[0])


In [ ]:
!echo $HF_API_KEY

### gemma hf

In [ ]:
from sentence_transformers import SentenceTransformer
import os
from dotenv import load_dotenv
print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))
# Hugging Face API key ortam değişkeninden çek
hf_token = os.getenv("HF_API_KEY")

# Modeli token ile yükle
model = SentenceTransformer(
    "google/embeddinggemma-300m",
    token=hf_token  # API key burada
)

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]

embeddings = model.encode(sentences)

# Benzerlik hesapla
similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import os
from dotenv import load_dotenv

# .env dosyasını yükle
load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env")

# Hugging Face API key
hf_token = os.getenv("HF_API_KEY")

# Modeli yükle (gated model ise token ver)
model = SentenceTransformer(
    "google/embeddinggemma-300m",
    token=hf_token
)

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]

# Embeddingleri al
embeddings = model.encode(sentences)

# Benzerlikleri cosine similarity ile hesapla
similarities = cosine_similarity(embeddings, embeddings)

print("Embeddings shape:", embeddings.shape)
print("Benzerlik matrisi:\n", similarities)


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("google/embeddinggemma-300m")
vec = model.encode(["test"])
print(vec.shape)  # (1, 784) olmalı

### chunk ederken cpu mu gpu mu kullanılıyor

In [ ]:
import torch
print("CUDA kullanılabilir mi? ", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU adı:", torch.cuda.get_device_name(0))
else:
    print("CPU kullanılıyor")


In [ ]:
import torch
from sentence_transformers import SentenceTransformer
import pandas as pd
import tiktoken
import semchunk
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import numpy as np
import uuid
from typing import List, Dict
import os
from dataclasses import dataclass
import json
from dotenv import load_dotenv

print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))

hf_token = os.getenv("HF_API_KEY")

# Konfigürasyon
@dataclass
class Config:
    # Google EmbeddingGemma ayarları
    MODEL_NAME: str = "google/embeddinggemma-300m"
    HF_TOKEN: str = hf_token

    # SemChunk ayarları
    TOKEN_SIZE: int = 512
    ENCODING_NAME: str = "cl100k_base"

    # Qdrant ayarları
    QDRANT_URL: str = "http://localhost:6333"
    COLLECTION_NAME: str = "gemma_semantic_chunks"
    DIMENSION: int = 768  # embeddinggemma-300m embedding boyutu

    # Dosya ayarları
    CSV_FILE: str = "/home/yapayzeka/ahsen_bulbul/data/yargitay_cleaned_2025-08-21.csv"
    BATCH_SIZE: int = 256

    # Cihaz ayarı
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"


class YargitaySemanticProcessor:
    def __init__(self, config: Config):
        self.config = config

        # SemChunk chunker
        self.encoding = tiktoken.get_encoding(config.ENCODING_NAME)
        self.chunker = semchunk.chunkerify(self.encoding, config.TOKEN_SIZE)

        # SentenceTransformer modeli (GPU/CPU seçimi burada yapılıyor)
        self.model = SentenceTransformer(
            config.MODEL_NAME,
            token=config.HF_TOKEN,
            device=config.DEVICE
        )
        print(f"✅ Model şu cihazda çalışıyor: {config.DEVICE}")

        # Qdrant client
        self.qdrant_client = QdrantClient(url=config.QDRANT_URL)

        print(f"✅ SemChunk hazır (Token boyutu: {config.TOKEN_SIZE})")
        print(f"✅ Google EmbeddingGemma modeli hazır")
        print(f"✅ Qdrant client hazır ({config.QDRANT_URL})")


### bir batch in embeddingleri oluştuktan sonra reduction yapıyo yani parça parça yapmış oluyor

In [ ]:
def create_embeddings(self, texts: List[str], batch_size: int = 100) -> List[List[float]]:
        all_embeddings = []
        total = len(texts)
        for i in range(0, total, batch_size):
            batch_texts = texts[i:i + batch_size]

            start_embed = time.time()

            batch_embeddings = self.model.encode(batch_texts, show_progress_bar=True)

            end_embed = time.time()
            print(f"Batch embedding süresi: {end_embed - start_embed:.2f} saniye")

            pca=PCA(n_components=512)
            batch_embeddings_reduced = pca.fit_transform(batch_embeddings) #default 768 olan embedding boyutunu 512ye düşürüyoruz -> PCA
            all_embeddings.extend(batch_embeddings_reduced.tolist())

            print(f"  🔹 Embedding oluşturuldu: {i + len(batch_texts)}/{total}")
        return all_embeddings

### model training

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# Modeli yükle
model = SentenceTransformer("google/embeddinggemma-300m")

# Küçük dataset
train_examples = [
    InputExample(texts=["araba", "otomobil"], label=1.0),
    InputExample(texts=["kedi", "köpek"], label=1.0),
    InputExample(texts=["masa", "sandalye"], label=1.0),
    InputExample(texts=["araba", "uçak"], label=0.0),
    InputExample(texts=["kedi", "araba"], label=0.0),
    InputExample(texts=["masa", "araba"], label=0.0),
]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)

# Loss fonksiyonu
train_loss = losses.CosineSimilarityLoss(model)

# Eğitim
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=10
)

# Test
emb1 = model.encode("araba")
emb2 = model.encode("otomobil")
emb3 = model.encode("kedi")

from sentence_transformers.util import cos_sim

print("Benzerlik (araba - otomobil):", cos_sim(emb1, emb2).item())
print("Benzerlik (araba - kedi):", cos_sim(emb1, emb3).item())


### bge 512 dimension with torch

In [ ]:
import torch
import torch.nn as nn

# Örnek 1024 boyutlu embedding
embedding_1024 = torch.randn(1, 1024)  # batch size 1

# Linear layer ile 1024 -> 512 boyuta düşürme
reduce_dim = nn.Linear(1024, 512)

# Boyut indirgeme işlemi
embedding_512 = reduce_dim(embedding_1024)

print("Orijinal boyut:", embedding_1024.shape)
print("Yeni boyut:", embedding_512.shape)



### bge normalization

In [ ]:
# main.py
# SemChunk + BGE-M3 + Qdrant Entegrasyon (Normalize edilmiş, reducer uyumlu, query_points kullanıyor)

import pandas as pd
import tiktoken
import semchunk
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from FlagEmbedding import BGEM3FlagModel
import numpy as np
import uuid
from typing import List, Dict, Any
import os
from dataclasses import dataclass
import json
from dotenv import load_dotenv
import torch
from torch import nn

print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))

# -------------------------
# Helper: normalize tensor rows (L2)
# -------------------------
def l2_normalize_tensor(t: torch.Tensor, eps: float = 1e-10) -> torch.Tensor:
    # t: (N, D)
    norm = torch.norm(t, dim=1, keepdim=True).clamp(min=eps)
    return t / norm

# Embed reducer (1024 -> 512)
class EmbedReducer(nn.Module):
    def __init__(self, input_dim: int = 1024, output_dim: int = 512):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear(x)

# -------------------------
# Config
# -------------------------
@dataclass
class Config:
    BGE_MODEL_NAME: str = "BAAI/bge-m3"
    USE_FP16: bool = True
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"
    TOKEN_SIZE: int = 512
    ENCODING_NAME: str = "cl100k_base"
    QDRANT_URL: str = "http://localhost:6333"
    COLLECTION_NAME: str = "yargitay_bge_m3_chunks"
    EMBEDDING_DIM: int = 512
    CSV_FILE: str = "/home/yapayzeka/ahsen_bulbul/data/cleaned10chunk.csv"
    BATCH_SIZE: int = 100
    DB_BATCH: int = 256

# -------------------------
# Processor
# -------------------------
class YargitaySemanticProcessor:
    def __init__(self, config: Config):
        self.config = config

        # Encoding & chunker
        self.encoding = tiktoken.get_encoding(config.ENCODING_NAME)
        self.chunker = semchunk.chunkerify(self.encoding, config.TOKEN_SIZE)

        # Model & reducer
        print(f"🔮 BGE-M3 yükleniyor: {config.BGE_MODEL_NAME} (device={config.DEVICE})")
        self.bge_model = BGEM3FlagModel(config.BGE_MODEL_NAME, use_fp16=config.USE_FP16, device=config.DEVICE)
        self.reducer = EmbedReducer(input_dim=1024, output_dim=self.config.EMBEDDING_DIM).to(config.DEVICE)

        # Qdrant
        self.qdrant_client = QdrantClient(url=config.QDRANT_URL)

        device_name = torch.cuda.get_device_name() if torch.cuda.is_available() else "CPU"
        print(f"✅ Hazır - Cihaz: {device_name}")

    # Test connection & print dense dim
    def test_bge_connection(self):
        try:
            test_text = ["Yargıtay 6. Hukuk Dairesi'nin ihtiyati tedbir kararı"]
            emb_res = self.bge_model.encode(test_text)
            dense = emb_res['dense_vecs'][0]
            print(f"✅ BGE-M3 test başarılı - Dense embedding boyutu: {len(dense)}")
            print(f"🔍 Sparse embedding mevcut: {'colbert_vecs' in emb_res}")
            return len(dense)
        except Exception as e:
            print(f"❌ BGE-M3 bağlantı hatası: {e}")
            return None

    def create_qdrant_collection(self, recreate: bool = False):
        collection_name = self.config.COLLECTION_NAME
        if recreate:
            try:
                self.qdrant_client.delete_collection(collection_name)
                print(f"🗑️ Eski koleksiyon silindi: {collection_name}")
            except Exception:
                pass

        try:
            existing = [c.name for c in self.qdrant_client.get_collections().collections]
            if collection_name not in existing:
                self.qdrant_client.create_collection(
                    collection_name=collection_name,
                    vectors_config=VectorParams(size=self.config.EMBEDDING_DIM, distance=Distance.COSINE)
                )
                print(f"✅ Koleksiyon oluşturuldu: {collection_name} (dim={self.config.EMBEDDING_DIM})")
            else:
                print(f"ℹ️ Koleksiyon zaten var: {collection_name}")
        except Exception as e:
            print(f"❌ Koleksiyon oluşturma hatası: {e}")
            raise

    def semantic_chunk_text(self, text: str, metadata: dict = None) -> List[Dict]:
        if not text or not text.strip():
            return []
        try:
            chunks = self.chunker(text)
            result = []
            for i, c in enumerate(chunks):
                if c.strip():
                    cd = {
                        'chunk_id': i,
                        'text': c.strip(),
                        'token_count': len(self.encoding.encode(c)),
                        'char_count': len(c)
                    }
                    if metadata:
                        cd.update(metadata)
                    result.append(cd)
            return result
        except Exception as e:
            print(f"❌ Chunking hatası: {e}")
            return []

    def create_embeddings_bge(self, texts: List[str], batch_size: int = None) -> List[List[float]]:
        batch_size = batch_size or self.config.BATCH_SIZE
        all_embeddings: List[List[float]] = []
        total = len(texts)
        print(f"🔮 BGE-M3 ile {total} metin işleniyor (batch_size={batch_size})...")

        for i in range(0, total, batch_size):
            batch_texts = texts[i:i + batch_size]
            try:
                # 1) embed (BGE-M3) -> dense (1024)
                emb_res = self.bge_model.encode(batch_texts)
                if isinstance(emb_res, dict) and 'dense_vecs' in emb_res:
                    dense = emb_res['dense_vecs']
                else:
                    dense = emb_res

                # 2) to tensor, device
                if not isinstance(dense, torch.Tensor):
                    dense_t = torch.tensor(dense, dtype=torch.float32, device=self.config.DEVICE)
                else:
                    dense_t = dense.to(self.config.DEVICE)

                # 3) reducer -> 512
                with torch.no_grad():
                    reduced = self.reducer(dense_t)

                # 4) normalize L2 (important for cosine)
                reduced = l2_normalize_tensor(reduced)

                # 5) append as python lists (cpu)
                all_embeddings.extend([v.cpu().tolist() for v in reduced])

                print(f"  📊 Batch işlendi: {i + len(batch_texts)}/{total}")

                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

            except Exception as e:
                print(f"❌ BGE-M3 Embedding hatası (batch {i//batch_size + 1}): {e}")
                # fallback zero vectors
                all_embeddings.extend([[0.0] * self.config.EMBEDDING_DIM for _ in batch_texts])

        return all_embeddings

    def process_csv_file(self, csv_path: str) -> List[Dict]:
        print(f"📄 CSV okunuyor: {csv_path}")
        try:
            df = pd.read_csv(csv_path)
            print(f"📊 {len(df)} satır yüklendi")
        except Exception as e:
            print(f"❌ CSV okuma hatası: {e}")
            return []

        text_column = next((c for c in ['rawText', 'chunk_text', 'text', 'content', 'metin'] if c in df.columns), None)
        if not text_column:
            print("❌ Ana metin sütunu bulunamadı")
            return []

        all_chunks = []
        for idx, row in df.iterrows():
            text = row.get(text_column, '')
            if not text or pd.isna(text):
                continue
            meta = {
                'original_index': idx,
                'esas_no': row.get('esasNo', ''),
                'karar_no': row.get('kararNo', ''),
                'daire': row.get('location', ''),
                'tarih': row.get('extractedDates', ''),
                'document_id': row.get('_id', ''),
            }
            chunks = self.semantic_chunk_text(str(text), meta)
            all_chunks.extend(chunks)
            if (idx + 1) % 5 == 0:
                print(f"  ✅ İşlenen satır: {idx + 1}/{len(df)} (Toplam chunk: {len(all_chunks)})")

        print(f"🧩 Toplam {len(all_chunks)} chunk oluşturuldu")
        return all_chunks

    def upload_to_qdrant(self, chunks: List[Dict]):
        if not chunks:
            print("❌ Yüklenecek chunk yok")
            return

        print(f"🚀 {len(chunks)} chunk Qdrant'a yükleniyor...")
        texts = [c['text'] for c in chunks]
        embeddings = self.create_embeddings_bge(texts)

        if len(embeddings) != len(chunks):
            print(f"❌ Embedding sayısı uyumsuz: {len(embeddings)} vs {len(chunks)}")
            return

        points = []
        for chunk, emb in zip(chunks, embeddings):
            points.append(PointStruct(id=str(uuid.uuid4()), vector=emb, payload=chunk))

        batch = self.config.DB_BATCH
        for i in range(0, len(points), batch):
            try:
                self.qdrant_client.upsert(collection_name=self.config.COLLECTION_NAME, points=points[i:i+batch])
                print(f"  ✅ Batch yüklendi: {min(i+batch, len(points))}/{len(points)}")
            except Exception as e:
                print(f"❌ Batch yükleme hatası: {e}")

        print("🎉 Yükleme tamamlandı!")

    # ---------- SEARCH (query_points + reducer + normalize) ----------
    def search_semantic(self, query: str, limit: int = 10, score_threshold: float = None) -> List[Dict]:
        try:
            emb_res = self.bge_model.encode([query])
            q_dense = emb_res['dense_vecs'] if isinstance(emb_res, dict) and 'dense_vecs' in emb_res else emb_res

            if not isinstance(q_dense, torch.Tensor):
                q_t = torch.tensor(q_dense, dtype=torch.float32, device=self.config.DEVICE)
            else:
                q_t = q_dense.to(self.config.DEVICE)

            with torch.no_grad():
                reduced_q = self.reducer(q_t)
                reduced_q = l2_normalize_tensor(reduced_q)

            query_vector = reduced_q[0].cpu().tolist()
            print(f"🔍 Query vector boyutu: {len(query_vector)} (hedef: {self.config.EMBEDDING_DIM})")

            # use query_points (recommended)
            qr = self.qdrant_client.query_points(
                collection_name=self.config.COLLECTION_NAME,
                query=query_vector,
                limit=limit,
                score_threshold=score_threshold
            )

            results = [{'score': p.score, 'payload': p.payload} for p in qr.points]
            print(f"📊 {len(results)} sonuç bulundu")
            return results

        except Exception as e:
            print(f"❌ Arama hatası: {e}")
            return []

    def advanced_search_with_filters(self, query: str, filters: Dict = None, limit: int = 10, score_threshold: float = None) -> List[Dict]:
        try:
            # prepare reduced + normalized query vector same as above
            emb_res = self.bge_model.encode([query])
            q_dense = emb_res['dense_vecs'] if isinstance(emb_res, dict) and 'dense_vecs' in emb_res else emb_res
            if not isinstance(q_dense, torch.Tensor):
                q_t = torch.tensor(q_dense, dtype=torch.float32, device=self.config.DEVICE)
            else:
                q_t = q_dense.to(self.config.DEVICE)

            with torch.no_grad():
                reduced_q = self.reducer(q_t)
                reduced_q = l2_normalize_tensor(reduced_q)

            query_vector = reduced_q[0].cpu().tolist()

            query_filter = None
            if filters:
                from qdrant_client.models import Filter, FieldCondition, MatchValue
                conditions = [FieldCondition(key=k, match=MatchValue(value=v)) for k, v in filters.items()]
                query_filter = Filter(must=conditions)

            qr = self.qdrant_client.query_points(
                collection_name=self.config.COLLECTION_NAME,
                query=query_vector,
                query_filter=query_filter,
                limit=limit,
                score_threshold=score_threshold
            )

            results = [{'score': p.score, 'payload': p.payload} for p in qr.points]
            print(f"📊 {len(results)} filtreli sonuç bulundu")
            return results

        except Exception as e:
            print(f"❌ Filtreli arama hatası: {e}")
            return []

    def get_collection_info(self) -> dict:
        try:
            info = self.qdrant_client.get_collection(self.config.COLLECTION_NAME)
            return {
                "collection_name": self.config.COLLECTION_NAME,
                "points_count": info.points_count,
                "vectors_count": info.vectors_count,
                "status": info.status,
                "embedding_model": "BGE-M3",
                "embedding_dim": self.config.EMBEDDING_DIM
            }
        except Exception as e:
            return {"error": str(e)}

# -------------------------
# Pipeline + main
# -------------------------
class YargitayPipeline:
    def __init__(self, config: Config):
        self.processor = YargitaySemanticProcessor(config)
        self.config = config

    def full_pipeline(self, csv_path: str = None):
        csv_path = csv_path or self.config.CSV_FILE
        print("🚀 Full pipeline başlıyor")
        emb_dim = self.processor.test_bge_connection()
        if not emb_dim:
            return False
        # recreate collection to ensure clean 512-dim index
        self.processor.create_qdrant_collection(recreate=True)
        chunks = self.processor.process_csv_file(csv_path)
        if not chunks:
            print("❌ Chunk bulunamadı")
            return False
        self.processor.upload_to_qdrant(chunks)
        info = self.processor.get_collection_info()
        print("\n📊 Koleksiyon Bilgileri:")
        print(json.dumps(info, indent=2, ensure_ascii=False))
        return True

    def interactive_search(self):
        print("\n🔎 İnteraktif arama başlatıldı")
        while True:
            print("\n1) Basit arama\n2) Filtreli arama\n3) Ana menü")
            ch = input("Seçiminiz (1-3): ").strip()
            if ch == "3":
                break
            if ch not in {"1", "2"}:
                print("❌ Geçersiz seçim")
                continue
            q = input("🔍 Arama metni (çıkmak için 'q'): ").strip()
            if q.lower() in {'q', 'quit', 'exit'}:
                break
            if not q:
                continue
            try:
                limit = int(input("Kaç sonuç? (default 5): ") or 5)
            except:
                limit = 5

            if ch == "1":
                # try with low threshold first for debugging
                results = self.processor.search_semantic(q, limit=limit, score_threshold=None)
            else:
                daire = input("Daire filtresi (örn: '6.HukukDairesi', boş = none): ").strip()
                filters = {'daire': daire} if daire else None
                results = self.processor.advanced_search_with_filters(q, filters=filters, limit=limit, score_threshold=None)

            if not results:
                print("❌ Sonuç bulunamadı")
                continue

            print(f"\n📋 {len(results)} sonuç:")
            for i, r in enumerate(results, 1):
                p = r['payload']
                print(f"\n{i}. Skor: {r['score']:.4f}")
                print(f"   Esas No: {p.get('esas_no','N/A')} | Karar No: {p.get('karar_no','N/A')}")
                print(f"   Daire: {p.get('daire','N/A')} | Tarih: {p.get('tarih','N/A')}")
                text_preview = (p.get('text','')[:300] + '...') if len(p.get('text','')) > 300 else p.get('text','')
                print(f"   Metin: {text_preview}")
                print("-"*60)

def main():
    config = Config(
        CSV_FILE="/home/yapayzeka/ahsen_bulbul/data/cleaned10chunk.csv",
        TOKEN_SIZE=512,
        QDRANT_URL="http://localhost:6333",
        COLLECTION_NAME="bge_normal_chunks",
        EMBEDDING_DIM=512,
        BATCH_SIZE=100
    )

    pipeline = YargitayPipeline(config)

    while True:
        print("\n" + "="*60)
        print("🏛️ YARGITAY BGE-M3 SEMANTİK SİSTEM")
        print("="*60)
        print("1) Tam pipeline çalıştır (CSV -> chunks -> embed -> qdrant)")
        print("2) İnteraktif arama")
        print("3) Koleksiyon bilgilerini göster")
        print("4) Çıkış")
        choice = input("Seçiminiz (1-4): ").strip()
        if choice == "1":
            csv_path = input(f"CSV yolu (enter ile default: {config.CSV_FILE}): ").strip() or config.CSV_FILE
            ok = pipeline.full_pipeline(csv_path)
            print("✅ Tamamlandı" if ok else "❌ Hata çıktı")
        elif choice == "2":
            pipeline.interactive_search()
        elif choice == "3":
            info = pipeline.processor.get_collection_info()
            print(json.dumps(info, indent=2, ensure_ascii=False))
        elif choice == "4":
            print("👋 Görüşürüz")
            break
        else:
            print("❌ Geçersiz seçim")

if __name__ == "__main__":
    try:
        from FlagEmbedding import BGEM3FlagModel
        print("✅ FlagEmbedding yüklü")
    except ImportError:
        print("❌ FlagEmbedding bulunamadı — pip install FlagEmbedding")
        raise SystemExit(1)
    main()


### gemma normalization

In [ ]:
# SemChunk + Google EmbeddingGemma + Qdrant Entegrasyon
# Yargıtay Kararları için Semantic Chunking Pipeline

from qdrant_client.models import VectorParams, Distance, PointStruct
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from sklearn.decomposition import PCA
from dataclasses import dataclass
from dotenv import load_dotenv
from typing import List, Dict
import numpy as np
import pandas as pd
import tiktoken
import semchunk
import uuid
import json
import time
import os
import torch
from torch import nn 
import torch.nn.functional as F

print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))

hf_token = os.getenv("HF_API_KEY")


class EmbedReducer(nn.Module):
    def __init__(self, input_dim=768, output_dim=512):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        # Xavier initialization
        nn.init.xavier_uniform_(self.linear.weight)
        nn.init.zeros_(self.linear.bias)

    def forward(self, x):
        # L2 normalization ekledik
        x = self.linear(x)
        return F.normalize(x, p=2, dim=-1)


# Konfigürasyon
@dataclass
class Config:
    # Google EmbeddingGemma ayarları
    MODEL_NAME: str = "google/embeddinggemma-300m"
    HF_TOKEN: str = hf_token

    # SemChunk ayarları
    TOKEN_SIZE: int = 512
    ENCODING_NAME: str = "cl100k_base"

    # Qdrant ayarları
    QDRANT_URL: str = "http://localhost:6333"
    COLLECTION_NAME: str = "gemma_semantic_chunks"
    DIMENSION: int = 512  #collection boyutu

    # Dosya ayarları
    CSV_FILE: str = "/home/yapayzeka/ahsen_bulbul/data/yargitay_cleaned_2025-08-21.csv"
    BATCH_SIZE: int = 100
    DB_BATCH_SIZE: int = 256

    
class YargitaySemanticProcessor:
    """Yargıtay kararları için semantic chunking ve vector search"""

    def __init__(self, config: Config):
        self.config = config

        # SemChunk chunker
        self.encoding = tiktoken.get_encoding(config.ENCODING_NAME)
        self.chunker = semchunk.chunkerify(self.encoding, config.TOKEN_SIZE)

        # SentenceTransformer modeli
        self.model = SentenceTransformer(
            config.MODEL_NAME,
            token=config.HF_TOKEN
        )

        # Reducer modeli - daha iyi initialization ile
        self.reducer = EmbedReducer(768, 512)
        self.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        self.reducer.to(self.device)

        # Qdrant client
        self.qdrant_client = QdrantClient(url=config.QDRANT_URL)

        print(f"✅ SemChunk hazır (Token boyutu: {config.TOKEN_SIZE})")
        print(f"✅ Google EmbeddingGemma modeli hazır")
        print(f"✅ Qdrant client hazır ({config.QDRANT_URL})")
        print(f"✅ Device: {self.device}")

    def create_qdrant_collection(self, recreate: bool = False):
        collection_name = self.config.COLLECTION_NAME
        if recreate:
            try:
                self.qdrant_client.delete_collection(collection_name)
                print(f"🗑️ Eski koleksiyon silindi: {collection_name}")
            except:
                pass

        try:
            collections = self.qdrant_client.get_collections().collections
            collection_names = [c.name for c in collections]

            if collection_name not in collection_names:
                self.qdrant_client.create_collection(
                    collection_name=collection_name,
                    vectors_config=VectorParams(
                        size=self.config.DIMENSION,
                        distance=Distance.COSINE
                    )
                )
                print(f"✅ Koleksiyon oluşturuldu: {collection_name}")
            else:
                print(f"ℹ️ Koleksiyon zaten var: {collection_name}")

        except Exception as e:
            print(f"❌ Koleksiyon oluşturma hatası: {e}")
            raise

    def semantic_chunk_text(self, text: str, metadata: dict = None) -> List[Dict]:
        if not text or text.strip() == "":
            return []

        chunks = self.chunker(text)
        result_chunks = []

        for i, chunk_text in enumerate(chunks):
            if chunk_text.strip():
                chunk_data = {
                    'chunk_id': i,
                    'text': chunk_text.strip(),
                    'token_count': len(self.encoding.encode(chunk_text)),
                    'char_count': len(chunk_text),
                }
                if metadata:
                    chunk_data.update(metadata)
                result_chunks.append(chunk_data)
        return result_chunks

    def create_embeddings(self, texts: List[str], batch_size: int = 100, target_dim: int = 512) -> List[List[float]]:
        all_embeddings = []
        total = len(texts)
        
        self.reducer.eval()  # Eval moduna al
        
        for i in range(0, total, batch_size):
            batch_texts = texts[i:i + batch_size]
            start_embed = time.time()
            
            # Embedding oluştur
            with torch.no_grad():
                batch_embeddings = self.model.encode(
                    batch_texts, 
                    show_progress_bar=True, 
                    convert_to_tensor=True, 
                    normalize_embeddings=True  # Bu önemli!
                ).to(self.device)
                
                # Boyut düşür ve normalize et
                reduced_vector = self.reducer(batch_embeddings)
                
                print("*" * 40)
                print(f"Original embedding shape: {batch_embeddings.shape}")
                print(f"Reduced vector shape: {reduced_vector.shape}")
                print(f"Vector norm check (should be ~1.0): {torch.norm(reduced_vector[0]).item():.4f}")
                print("*" * 40)
                
                # CPU'ya taşı ve listeye çevir
                all_embeddings.extend(reduced_vector.cpu().tolist())
            
            end_embed = time.time()
            print(f"Batch embedding süresi: {end_embed - start_embed:.2f} saniye")
            print(f"  🔹 Embedding oluşturuldu: {i + len(batch_texts)}/{total}")

        return all_embeddings

    def process_csv_file(self, csv_path: str) -> List[Dict]:
        try:
            df = pd.read_csv(csv_path)
        except Exception as e:
            print(f"❌ CSV okuma hatası: {e}")
            return []

        all_chunks = []
        total_rows = len(df)
        print(f"📄 Toplam {total_rows} satır işlenecek")

        for idx, row in df.iterrows():
            text = row.get('rawText', '') or row.get('text', '')
            if not text or pd.isna(text):
                continue

            metadata = {
                'original_index': idx,
                'esas_no': row.get('esasNo', ''),
                'karar_no': row.get('kararNo', ''),
                'daire': row.get('location', ''),
                'tarih': row.get('extractedDates', '')
            }

            chunks = self.semantic_chunk_text(str(text), metadata)
            all_chunks.extend(chunks)

            if (idx + 1) % 50 == 0 or (idx + 1) == total_rows:
                print(f"  ✅ İşlenen satır: {idx + 1}/{total_rows} (Toplam chunk: {len(all_chunks)})")

        print(f"🧩 Toplam {len(all_chunks)} chunk oluşturuldu")
        return all_chunks

    def upload_to_qdrant(self, chunks: List[Dict]):
        if not chunks:
            print("❌ Yüklenecek chunk yok")
            return

        texts = [chunk['text'] for chunk in chunks]
        print("🔮 Embedding'ler oluşturuluyor...")
        embeddings = self.create_embeddings(texts)

        points = [
            PointStruct(
                id=str(uuid.uuid4()),
                vector=embedding,
                payload=chunk
            ) for chunk, embedding in zip(chunks, embeddings)
        ]

        batch_size = self.config.DB_BATCH_SIZE
        total_points = len(points)
        print(f"🚀 {total_points} chunk Qdrant'a yükleniyor ({batch_size} batch size)")

        for i in range(0, total_points, batch_size):
            batch = points[i:i + batch_size]
            try:
                start_upload = time.time()

                self.qdrant_client.upsert(
                    collection_name=self.config.COLLECTION_NAME,
                    points=batch
                )

                end_upload = time.time()
                print(f"batch Qdrant yükleme süresi: {end_upload - start_upload:.2f} saniye")
                print(f"  ✅ Batch yüklendi: {min(i + batch_size, total_points)}/{total_points}")
            except Exception as e:
                print(f"❌ Batch yükleme hatası: {e}")

        print(f"🎉 {total_points} chunk Qdrant'a yüklendi!")

    def search_semantic(self, query: str, limit: int = 10, score_threshold: float = 0.3) -> List[Dict]:
        """Düzeltilmiş search metodu - daha düşük threshold ile"""
        
        self.reducer.eval()
        
        with torch.no_grad():
            # Query embedding çıkar (normalize edilmiş)
            query_embedding = self.model.encode(
                [query], 
                convert_to_tensor=True, 
                normalize_embeddings=True  # Bu önemli!
            ).to(self.device)
            
            # Boyutu 512'ye düşür ve normalize et
            reduced_query_embedding = self.reducer(query_embedding)
            
            print(f"🔍 Query vector boyutu: {reduced_query_embedding.shape}")
            print(f"🔍 Query vector norm: {torch.norm(reduced_query_embedding[0]).item():.4f}")
            
            # CPU'ya taşı ve numpy array'e çevir
            query_vector = reduced_query_embedding[0].cpu().numpy().tolist()
        
        print(f"🔍 Final query vector boyutu: {len(query_vector)} (hedef: {self.config.DIMENSION})")
        
        try:
            # Qdrant araması - daha düşük threshold ile
            search_results = self.qdrant_client.search(
                collection_name=self.config.COLLECTION_NAME,
                query_vector=query_vector,
                limit=limit,
                score_threshold=score_threshold
            )
            
            print(f"🔍 Ham arama sonucu sayısı: {len(search_results)}")
            
            # Eğer hiç sonuç yoksa, threshold'suz deneyelim
            if not search_results:
                print("⚠️ Threshold ile sonuç yok, threshold'suz deneniyor...")
                search_results = self.qdrant_client.search(
                    collection_name=self.config.COLLECTION_NAME,
                    query_vector=query_vector,
                    limit=limit
                )
                print(f"🔍 Threshold'suz sonuç sayısı: {len(search_results)}")
                
                if search_results:
                    print(f"🔍 En yüksek score: {search_results[0].score:.4f}")
                    print(f"🔍 En düşük score: {search_results[-1].score:.4f}")
            
            results = [{'score': p.score, 'payload': p.payload} for p in search_results]
            return results
            
        except Exception as e:
            print(f"❌ Arama hatası: {e}")
            return []

    def debug_search_issue(self, query: str):
        """Arama problemini debug et"""
        print(f"🔧 Debug başlıyor - Query: '{query}'")
        
        # 1. Koleksiyon durumunu kontrol et
        info = self.get_collection_info()
        print(f"📊 Koleksiyon durumu: {info}")
        
        if info.get('points_count', 0) == 0:
            print("❌ Koleksiyonda hiç point yok!")
            return
        
        # 2. Query embedding oluştur
        self.reducer.eval()
        with torch.no_grad():
            query_embedding = self.model.encode([query], convert_to_tensor=True, normalize_embeddings=True).to(self.device)
            reduced_query = self.reducer(query_embedding)
            query_vector = reduced_query[0].cpu().numpy().tolist()
            
        print(f"🔍 Query vector shape: {len(query_vector)}")
        print(f"🔍 Query vector norm: {np.linalg.norm(query_vector):.4f}")
        print(f"🔍 Query vector sample: {query_vector[:5]}")
        
        # 3. Çok düşük threshold ile ara
        try:
            results = self.qdrant_client.search(
                collection_name=self.config.COLLECTION_NAME,
                query_vector=query_vector,
                limit=5,
                score_threshold=0.0  # Çok düşük threshold
            )
            
            print(f"🔍 Score threshold 0.0 ile bulunan sonuç: {len(results)}")
            if results:
                for i, r in enumerate(results):
                    print(f"  {i+1}. Score: {r.score:.6f}")
                    text = r.payload.get('text', '')[:100]
                    print(f"     Text preview: {text}...")
            
        except Exception as e:
            print(f"❌ Debug search hatası: {e}")
    
    def fix_dimension_mismatch(self):
        """Dimension uyumsuzluğunu düzelt"""
        print("🔧 Dimension uyumsuzluğu düzeltiliyor...")
        response = input("⚠️ Bu işlem koleksiyonu silecek. Devam? (y/N): ")
        if response.lower() != 'y':
            return False
            
        try:
            self.qdrant_client.delete_collection(self.config.COLLECTION_NAME)
            print("✅ Koleksiyon silindi")
            return True
        except Exception as e:
            print(f"❌ Silme hatası: {e}")
            return False

    def get_collection_info(self) -> dict:
        """Koleksiyon bilgilerini al"""
        try:
            info = self.qdrant_client.get_collection(self.config.COLLECTION_NAME)
            return {
                "collection_name": self.config.COLLECTION_NAME,
                "points_count": info.points_count,
                "vectors_count": info.vectors_count,
                "status": info.status
            }
        except Exception as e:
            return {"error": str(e)}


# Pipeline sınıfı
class YargitayPipeline:
    def __init__(self, config: Config):
        self.processor = YargitaySemanticProcessor(config)
        self.config = config

    def full_pipeline(self, csv_path: str = None):
        csv_path = csv_path or self.config.CSV_FILE
        print("🚀 Yargıtay Semantic Pipeline Başlıyor")

        total_start = time.time()

        # Koleksiyon oluşturma
        self.processor.create_qdrant_collection(recreate=True)

        # CSV işlemleri ve chunk oluşturma
        chunk_start = time.time()
        chunks = self.processor.process_csv_file(csv_path)
        chunk_end = time.time()

        if not chunks:
            print("❌ İşlenecek chunk bulunamadı")
            return False

        # Embedding oluşturma ve Qdrant yükleme
        upload_start = time.time()
        self.processor.upload_to_qdrant(chunks)
        upload_end = time.time()

        total_end = time.time()

        # Toplam istatistikler
        info = self.processor.get_collection_info()
        print("\n📊 Pipeline Süreleri ve İstatistikler:")
        print(json.dumps({
            "collection_name": self.config.COLLECTION_NAME,
            "points_uploaded": info.get("points_count", 0),
            "chunk_creation_time_s": round(chunk_end - chunk_start, 2),
            "embedding_and_upload_time_s": round(upload_end - upload_start, 2),
            "total_pipeline_time_s": round(total_end - total_start, 2)
        }, indent=2, ensure_ascii=False))

        return True

    def interactive_search(self):
        print("🔍 İnteraktif Arama Başlatıldı")
        
        # Önce koleksiyon durumunu kontrol et
        info = self.processor.get_collection_info()
        print(f"📊 Koleksiyon Durumu: {json.dumps(info, indent=2, ensure_ascii=False)}")
        
        while True:
            query = input("\n🔍 Arama metni (çıkmak için 'q'): ").strip()
            if query.lower() in ['q', 'quit', 'exit']:
                break
                
            if not query:
                print("❌ Boş sorgu, tekrar deneyin")
                continue
            
            limit_input = input("Kaç sonuç? (default 5): ").strip()
            try:
                limit = int(limit_input) if limit_input else 5
                limit = max(1, min(limit, 50))
            except ValueError:
                print("❌ Geçersiz sayı, varsayılan 5 kullanılıyor")
                limit = 5

            score_input = input("Minimum score? (default 0.3): ").strip()
            try:
                score_threshold = float(score_input) if score_input else 0.3
                score_threshold = max(0.0, min(score_threshold, 1.0))
            except ValueError:
                print("❌ Geçersiz score, varsayılan 0.3 kullanılıyor")
                score_threshold = 0.3

            print(f"\n{'='*60}")
            results = self.processor.search_semantic(query, limit=limit, score_threshold=score_threshold)
            
            if not results:
                print("❌ Hiç sonuç bulunamadı.")
                print("💡 Öneriler:")
                print("   - Score threshold'u düşürün (0.03 veya 0.05)")
                print("   - Debug modunu çalıştırın (ana menüden 4)")
                print("   - Daha spesifik kelimeler kullanın")
            else:
                print(f"\n📋 {len(results)} Sonuç Bulundu:")
                for i, r in enumerate(results, 1):
                    payload = r['payload']
                    text_preview = payload.get('text', '')[:200] + "..." if len(payload.get('text', '')) > 200 else payload.get('text', '')
                    print(f"\n{i}. 📊 Score: {r['score']:.4f}")
                    print(f"   📋 Esas No: {payload.get('esas_no', 'N/A')}")
                    print(f"   📋 Karar No: {payload.get('karar_no', 'N/A')}")
                    print(f"   📋 Daire: {payload.get('daire', 'N/A')}")
                    print(f"   📋 Tarih: {payload.get('tarih', 'N/A')}")
                    print(f"   📄 Metin: {text_preview}")
                    print(f"   {'─'*50}")
            
            print(f"{'='*60}")


# Main fonksiyon
def main():
    config = Config(
        CSV_FILE="/home/yapayzeka/ahsen_bulbul/data/yargitay_cleaned_2025-08-21.csv",
        TOKEN_SIZE=512,
        QDRANT_URL="http://localhost:6333",
        COLLECTION_NAME="google_gemma_chunks",
        DIMENSION=512
    )

    pipeline = YargitayPipeline(config)

    while True:
        print("\n" + "="*60)
        print("1. Full pipeline çalıştır")
        print("2. Arama yap") 
        print("3. Koleksiyon bilgisi")
        print("4. 🔧 Debug arama sorunu")
        print("5. 🗑️ Dimension uyumsuzluğunu düzelt")
        print("6. Çıkış")
        print("="*60)
        
        choice = input("Seçim: ").strip()
        
        if choice == "1":
            csv_path = input(f"CSV dosya yolu (Enter: {config.CSV_FILE}): ").strip() or config.CSV_FILE
            pipeline.full_pipeline(csv_path)
            
        elif choice == "2":
            pipeline.interactive_search()
            
        elif choice == "3":
            info = pipeline.processor.get_collection_info()
            print("\n📊 Koleksiyon Bilgileri:")
            print(json.dumps(info, indent=2, ensure_ascii=False))
            
        elif choice == "4":
            query = input("Debug için test query (Enter: 'test'): ").strip() or "test"
            pipeline.processor.debug_search_issue(query)
            
        elif choice == "5":
            if pipeline.processor.fix_dimension_mismatch():
                print("✅ Koleksiyon sıfırlandı. Şimdi '1' seçeneği ile veriyi yeniden yükleyin.")
            else:
                print("❌ İşlem iptal edildi")
                
        elif choice == "6":
            break
        else:
            print("❌ Geçersiz seçim")


if __name__ == "__main__":
    main()

True
✅ SemChunk hazır (Token boyutu: 512)
✅ Google EmbeddingGemma modeli hazır
✅ Qdrant client hazır (http://localhost:6333)
✅ Device: cuda:0

1. Full pipeline çalıştır
2. Arama yap
3. Koleksiyon bilgisi
4. 🔧 Debug arama sorunu
5. 🗑️ Dimension uyumsuzluğunu düzelt
6. Çıkış
🔍 İnteraktif Arama Başlatıldı
📊 Koleksiyon Durumu: {
  "collection_name": "google_normal_chunks",
  "points_count": 80345,
  "vectors_count": null,
  "status": "green"
}

🔍 Query vector boyutu: torch.Size([1, 512])
🔍 Query vector norm: 1.0000
🔍 Final query vector boyutu: 512 (hedef: 512)
🔍 Ham arama sonucu sayısı: 0
⚠️ Threshold ile sonuç yok, threshold'suz deneniyor...
🔍 Threshold'suz sonuç sayısı: 5
🔍 En yüksek score: 0.1165
🔍 En düşük score: 0.1068

📋 5 Sonuç Bulundu:

1. 📊 Score: 0.1165
   📋 Esas No: 2024/1564 E.
   📋 Karar No: 2024/2633 K.
   📋 Daire: 1.HukukDairesisi
   📋 Tarih: 08.04.2008,20.09.2010,18.03.2014,27.11.1937,01.04.2024
   📄 Metin: Tapu kaydının uygulanmasında tüm komşu parsel tutanak ve dayanaklar

/tmp/ipykernel_1560741/860841019.py:278: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = self.qdrant_client.search(
/tmp/ipykernel_1560741/860841019.py:290: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = self.qdrant_client.search(



🔍 Query vector boyutu: torch.Size([1, 512])
🔍 Query vector norm: 1.0000
🔍 Final query vector boyutu: 512 (hedef: 512)
🔍 Ham arama sonucu sayısı: 0
⚠️ Threshold ile sonuç yok, threshold'suz deneniyor...
🔍 Threshold'suz sonuç sayısı: 5
🔍 En yüksek score: 0.1165
🔍 En düşük score: 0.1068

📋 5 Sonuç Bulundu:

1. 📊 Score: 0.1165
   📋 Esas No: 2024/1564 E.
   📋 Karar No: 2024/2633 K.
   📋 Daire: 1.HukukDairesisi
   📋 Tarih: 08.04.2008,20.09.2010,18.03.2014,27.11.1937,01.04.2024
   📄 Metin: Tapu kaydının uygulanmasında tüm komşu parsel tutanak ve dayanaklarından yararlanılarak tapu kayıt uygulaması denetlenmemiş, tapu kaydının miktar fazlası bulunup bulunmadığı incelenmemiş, keşif yapılm...
   ──────────────────────────────────────────────────

2. 📊 Score: 0.1129
   📋 Esas No: 2021/17062 E.
   📋 Karar No: 2024/730 K.
   📋 Daire: 4.HukukDairesisi
   📋 Tarih: 24.12.2018,02.05.2018,02.05.2018,23.01.2008,22.01.2024
   📄 Metin: İcra Müdürlüğü'nün 2008/465 (2017/3329 E.) sayılı icra dosyasından tak